# Semantic Text Similarity
Este modelo utiliza gensim para convertir pares de vectores + puntuaciones en vectores (word embeddings).
Dado un dataset, infiere la puntuación de similitud entre ambas frases.

In [1]:
# Requisitos
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np

In [2]:
# Tipado
from typing import Tuple, List

In [3]:
# Cargar stopwords en Catalan
# STOPWORDS_CA = {"a", "abans", "ací", "ah", "així", "això", "al", "aleshores", "algun", "alguna", "algunes", "alguns", "alhora", "allà", "allí", "allò", "als", "altra", "altre", "altres", "amb", "ambdues", "ambdós", "anar", "ans", "apa", "aquell", "aquella", "aquelles", "aquells", "aquest", "aquesta", "aquestes", "aquests", "aquí", "baix", "bastant", "bé", "cada", "cadascuna", "cadascunes", "cadascuns", "cadascú", "com", "consegueixo", "conseguim", "conseguir", "consigueix", "consigueixen", "consigueixes", "contra", "d'un", "d'una", "d'unes", "d'uns", "dalt", "de", "del", "dels", "des", "des de", "després", "dins", "dintre", "donat", "doncs", "durant", "e", "eh", "el", "elles", "ells", "els", "em", "en", "encara", "ens", "entre", "era", "erem", "eren", "eres", "es", "esta", "estan", "estat", "estava", "estaven", "estem", "esteu", "estic", "està", "estàvem", "estàveu", "et", "etc", "ets", "fa", "faig", "fan", "fas", "fem", "fer", "feu", "fi", "fins", "fora", "gairebé", "ha", "han", "has", "haver", "havia", "he", "hem", "heu", "hi", "ho", "i", "igual", "iguals", "inclòs", "ja", "jo", "l'hi", "la", "les", "li", "li'n", "llarg", "llavors", "m'he", "ma", "mal", "malgrat", "mateix", "mateixa", "mateixes", "mateixos", "me", "mentre", "meu", "meus", "meva", "meves", "mode", "molt", "molta", "moltes", "molts", "mon", "mons", "més", "n'he", "n'hi", "ne", "ni", "no", "nogensmenys", "només", "nosaltres", "nostra", "nostre", "nostres", "o", "oh", "oi", "on", "pas", "pel", "pels", "per", "per que", "perquè", "però", "poc", "poca", "pocs", "podem", "poden", "poder", "podeu", "poques", "potser", "primer", "propi", "puc", "qual", "quals", "quan", "quant", "que", "quelcom", "qui", "quin", "quina", "quines", "quins", "què", "s'ha", "s'han", "sa", "sabem", "saben", "saber", "sabeu", "sap", "saps", "semblant", "semblants", "sense", "ser", "ses", "seu", "seus", "seva", "seves", "si", "sobre", "sobretot", "soc", "solament", "sols", "som", "son", "sons", "sota", "sou", "sóc", "són", "t'ha", "t'han", "t'he", "ta", "tal", "també", "tampoc", "tan", "tant", "tanta", "tantes", "te", "tene", "tenim", "tenir", "teniu", "teu", "teus", "teva", "teves", "tinc", "ton", "tons", "tot", "tota", "totes", "tots", "un", "una", "unes", "uns", "us", "va", "vaig", "vam", "van", "vas", "veu", "vosaltres", "vostra", "vostre", "vostres", "érem", "éreu", "és", "éssent", "últim", "ús"}
STOPWORDS_CA = {"a", "al", "el", "la", "els", "les", "de", "un", "una", "algun", "alguna", }

In [4]:
# Definir función de pre-procesado
def preprocess(sentence: str) -> List[str]:
    preprocessed = simple_preprocess(sentence)
    preprocessed = [token for token in preprocessed if token not in STOPWORDS_CA]
    return preprocessed

In [6]:
# Modelos pre-entrenados
WV_MODEL_PATH = "/Users/salva/Downloads/cc.ca.300.bin.gz"
# from gensim.models import Word2Vec
# wv_model = Word2Vec.load('path_to_word2vec_model').wv
from gensim.models import fasttext
wv_model = fasttext.load_facebook_vectors(WV_MODEL_PATH)

In [0]:
# Heu de guardar el model en un format compatible
wv_model.save('cc.ca.gensim.bin')

In [5]:
# Llavors podeu carregar el model com a mmap
from gensim.models.fasttext import FastTextKeyedVectors
wv_model = FastTextKeyedVectors.load('cc.ca.gensim.bin', mmap='r')

In [6]:
# Ejemplo de 10 pares de oraciones con puntuación de similitud asociada
input_pairs = [
    ('M\'agrada el futbol', 'Disfruto veient partits de futbol', 4),
    ('El cel està despejat', 'Fa un dia bonic', 4.5),
    ('M\'encanta viatjar', 'Explorar nous llocs és una passió', 3.5),
    ('Prefereixo l\'estiu', 'No m\'agrada el fred de l\'hivern', 2.5),
    ('Tinc gana', 'Què hi ha per sopar?', 2),
    ('La música em relaxa', 'Escoltar música és una teràpia', 3),
    ('El llibre és emocionant', 'No puc deixar de llegir-lo', 4),
    ('M\'agrada la pizza', 'És el meu menjar preferit', 4.5),
    ('Estic cansat', 'Necessito fer una migdiada', 1.5),
    ('Avui fa molta calor', 'És un dia sofocant', 3.5)
    ]

In [18]:
# Datos reales
TRAIN_DATA_FILE: str = "/Users/salva/Downloads/train.tsv"
import pandas as pd
tsv_data = pd.read_csv(TRAIN_DATA_FILE, sep='\t', header=None, usecols=[1, 2, 3])
input_pairs = tsv_data.values.tolist()

In [19]:
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [preprocess(sentence_1) for sentence_1, _, _ in input_pairs]
sentences_2_preproc = [preprocess(sentence_2) for _, sentence_2, _ in input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)

In [20]:
print(sentence_pairs[0])

(['atorga', 'per', 'primer', 'cop', 'mencions', 'encarna', 'sanahuja', 'inclusió', 'perspectiva', 'gènere', 'en', 'docència', 'universitària'], ['creen', 'menció', 'encarna', 'sanahuja', 'inclusió', 'perspectiva', 'gènere', 'en', 'docència', 'universitària'])


In [21]:
# Cálculo de los pesos TF-IDF para las oraciones pre-procesadas
corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
modelo_tfidf = TfidfModel(corpus)

In [217]:
def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc)
    tf_idf = tf_idf_model[bow]
    vectors, weights = [], []
    for word_index, weight in tf_idf:
        word = dictionary.get(word_index)
        if word in wv_model:
            vectors.append(wv_model[word])
            weights.append(weight)
    return vectors, weights

def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        tf_idf_model: TfidfModel = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    """
    Mapea los tripletes de oraciones a listas de (x, y), (pares de vectores, score)
    :param sentence_pairs:
    :param dictionary:
    :param tf_idf_model:
    :return:
    """
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess(sentence_1)
        sentence_2_preproc = preprocess(sentence_2)
        # Si usamos TF-IDF
        if tf_idf_model is not None:
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            vector1 = np.average(vectors1, weights=weights1, axis=0, )
            vector2 = np.average(vectors2, weights=weights2, axis=0, )
        else:
            # Cálculo del promedio de los word embeddings
            vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]
            vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
            vector1 = np.mean(vectors1, axis=0)
            vector2 = np.mean(vectors2, axis=0)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [218]:
# Imprimir los pares de vectores y la puntuación de similitud asociada
mapped = map_pairs(input_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
for vectors, similitud in mapped:
    print(f"Pares de vectores: {vectors[0].shape}, {vectors[1].shape}")
    print(f"Puntuación de similitud: {similitud}")
    print()

Pares de vectores: (300,), (300,)
Puntuación de similitud: 3.5

Pares de vectores: (300,), (300,)
Puntuación de similitud: 1.25

Pares de vectores: (300,), (300,)
Puntuación de similitud: 3.67

Pares de vectores: (300,), (300,)
Puntuación de similitud: 2.25

Pares de vectores: (300,), (300,)
Puntuación de similitud: 2.0

Pares de vectores: (300,), (300,)
Puntuación de similitud: 2.75

Pares de vectores: (300,), (300,)
Puntuación de similitud: 2.67

Pares de vectores: (300,), (300,)
Puntuación de similitud: 2.5

Pares de vectores: (300,), (300,)
Puntuación de similitud: 2.5

Pares de vectores: (300,), (300,)
Puntuación de similitud: 3.0

Pares de vectores: (300,), (300,)
Puntuación de similitud: 3.0

Pares de vectores: (300,), (300,)
Puntuación de similitud: 1.0

Pares de vectores: (300,), (300,)
Puntuación de similitud: 2.0

Pares de vectores: (300,), (300,)
Puntuación de similitud: 4.0

Pares de vectores: (300,), (300,)
Puntuación de similitud: 3.0

Pares de vectores: (300,), (300,)
P

In [219]:
# Definir el Modelo
import tensorflow as tf

def build_and_compile_model(hidden_size: int = 128, embedding_size: int = 300, learning_rate: float = 0.001) -> tf.keras.Model:
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Capa oculta
    first_projection = tf.keras.layers.Dense(
        embedding_size,
        # activation='tanh',
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 = first_projection(input_1)
    projected_2 = first_projection(input_2)

    # Compute the cosine distance
    projected_1 = tf.linalg.l2_normalize(projected_1, axis=1, )
    projected_2 = tf.linalg.l2_normalize(projected_2, axis=1, )
    output = 2.5 * (1.0 + tf.reduce_sum(projected_1 * projected_2, axis=1, ))

    # Definir el modelo con las capas de entrada y salida
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compilar el modelo
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))

    return model

In [220]:
# Definir constantes de entrenamiento
batch_size: int = 64
num_epochs: int = 64
train_val_split: float = 0.8

In [221]:
# Obtener x_train e y_train
train_slice: int = int(len(mapped) * train_val_split)

def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    """
    Otiene las matrices X_1 (N x d) , X_2 (N x d), e Y (n) a partir de listas de parejas de vectores de oraciones - Listas de (d, d, 1)
    :param pair_list:
    :return:
    """
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, )

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped[:train_slice])
x_val, y_val = pair_list_to_x_y(mapped[train_slice:])

In [222]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [223]:
# Construir y compilar el modelo
model = build_and_compile_model()
# tf.keras.utils.plot_model(model, show_shapes=True, show_layer_activations=True, )
print(model.summary())
# Entrenar el modelo
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_63 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 input_64 (InputLayer)          [(None, 300)]        0           []                               
                                                                                                  
 dense_36 (Dense)               (None, 300)          90300       ['input_63[0][0]',               
                                                                  'input_64[0][0]']               
                                                                                                  
 tf.math.l2_normalize_18 (TFOpL  (None, 300)         0           ['dense_36[0][0]']        

In [224]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
from scipy.stats import pearsonr
from scipy import spatial
val = mapped[train_slice:]
y_pred_baseline = []
for (v1, v2), dist in val:
    d = 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(np.array(y_pred_baseline), y_val.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")

Correlación de Pearson: 0.2933942210300401


In [225]:
from scipy.stats import pearsonr
# Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
y_pred: tf.RaggedTensor = model.predict(x_val)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(y_pred.flatten(), y_val.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}")


13/13 [==============================] - 0s 9ms/step
Correlación de Pearson: 0.3325139706428133
